This script generates the ETCCDI temperature indices from a single historical run of CESM2-WACCM daily temperature maxima (TX) and temperature minima (TN).

Outputs one file per index:

* TXX  
* TXN  
* TNN   
* TNX  
* TN90  
* TN10  
* TX90  
* TX10  
* SU   
* TR  
* ID 
* FD  

GSL Growing season length is not computed here. Script yet to be developed.  
Warm spell and cold spell durations are not computed. Script yet to be developed.  

__NOTE__ percentile days are with respect to the annual distribution. i.e. number of days exceeding the 90th percentile of the annual distribution of temperatures. Climpact calculates a percentage of days higher than the calendar day threshold (i.e. with respect to seasonal distribution). This is to come.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np # data arrays
import xarray as xr # data array manipulation
import pandas as pd
import datetime as dt
import os
import glob

Define the temperature functions

In [3]:
from importlib import reload
import config
import tempex_func
import utils #pyclimdex by B Groenks


Read in each of the datasets in turn, calculate indices, then write the output to netcdf.

In [4]:
iDir = "/glade/derecho/scratch/maritye/"
# preprocessed files
otnDir = "/glade/work/maritye/Data/ARISE-SAI/ETCCDI/Historical/TREFHTMN/"
otxDir = '/glade/work/maritye/Data/ARISE-SAI/ETCCDI/Historical/TREFHTMX/'

Create standard information to include with each data set

In [5]:
years = np.arange(1980,2014)
dates= '1980-2014'
#member=range(10)
lat = np.linspace(-90,90,num=192)
lon = np.linspace(0,358.75,num=288)
#dims = ('member','year', 'lat', 'lon')
#coords = dict(member = member, year=years, lat=lat, lon=lon)
dims = ('year', 'lat', 'lon')
coords = dict(year=years, lat=lat, lon=lon)
attribs = dict(description='Temperature Extreme Indices based on ETCCDI definitions. MCB scenarios with 5% global seeding', 
                history='Created by Mari Tye November 2023.' ),


read in thresholds from SSP245 take the mean of the ensemble as the threshold for these runs.

In [7]:
%%time 
with xr.open_dataset('/glade/work/maritye/Data/ARISE-SAI/ETCCDI/Historical/TREFHTMN/b.e21.BWHISTcmip6.f09_g17.CMIP6-historical-WACCM.1980_2014.001.cam.h1.QN10.1981-2010.nc') as dn10:
    print('dn10')

with xr.open_dataset('/glade/work/maritye/Data/ARISE-SAI/ETCCDI/Historical/TREFHTMN/b.e21.BWHISTcmip6.f09_g17.CMIP6-historical-WACCM.1980_2014.001.cam.h1.QN90.1981-2010.nc') as dn90:
    print('dn90')
    
with xr.open_dataset('/glade/work/maritye/Data/ARISE-SAI/ETCCDI/Historical/TREFHTMX/b.e21.BWHISTcmip6.f09_g17.CMIP6-historical-WACCM.1980_2014.001.cam.h1.QX90.1981-2010.nc') as dsx90:
    print('dx90')

with xr.open_dataset('/glade/work/maritye/Data/ARISE-SAI/ETCCDI/Historical/TREFHTMX/b.e21.BWHISTcmip6.f09_g17.CMIP6-historical-WACCM.1980_2014.001.cam.h1.QX10.1981-2010.nc') as dsx10:
    print('dx10')

dn10
dn90
dx90
dx10
CPU times: user 11.2 ms, sys: 0 ns, total: 11.2 ms
Wall time: 11.2 ms


In [8]:
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client

In [9]:
# Create a PBS cluster object
cluster = PBSCluster(
    job_name = 'dask-wk23-hpc',
    cores = 1,
    memory = '4GiB',
    processes = 1,
    local_directory = '/local_scratch/pbs.$PBS_JOBID/dask/spill',
    resource_spec = 'select=1:ncpus=1:mem=4GB',
    queue = 'casper',
    walltime = '30:00',
    interface = 'ext'
)

In [11]:
# Create the client to load the Dashboard
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/negins/Backup/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/negins/Backup/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.83:44629,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/negins/Backup/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [12]:
num_workers = 6 
cluster.scale(num_workers)

client.wait_for_workers(num_workers)

## Daily minimum temperatures

Create some blank arrays to fill with each index for each ensemble member. Write these out at the end with netcdf

In [8]:
TNX = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='TNX')
TNN = xr.DataArray(None, coords=coords,  dims=dims, attrs=attribs, name='TNN')
TN90 = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='TN90')
TN10 = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='TN10')
TR = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='TR')
FD = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='FD')

list the files - glob includes the filepath

In [13]:
files = glob.glob(iDir + '*.TREFHTMN.*')

In [14]:
files

['/glade/derecho/scratch/maritye/b.e21.BWHISTcmip6.f09_g17.CMIP6-historical-WACCM.1980_2014.001.cam.h1.TREFHTMN.19800101-20141231.nc']

In [16]:
%%time
ds = xr.open_mfdataset(files, parallel=True)

CPU times: user 4.46 s, sys: 176 ms, total: 4.64 s
Wall time: 53.9 s


In [17]:
ds

<xarray.Dataset>
Dimensions:   (lat: 192, lon: 288, time: 12776)
Coordinates:
  * lat       (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon       (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time      (time) object 1980-01-01 00:00:00 ... 2014-12-31 00:00:00
Data variables:
    TREFHTMN  (time, lat, lon) float32 dask.array<chunksize=(12776, 192, 288), meta=np.ndarray>

In [18]:
tref_min = ds['TREFHTMN']
tref_min

<xarray.DataArray 'TREFHTMN' (time: 12776, lat: 192, lon: 288)>
dask.array<open_dataset-d7a9874df7c91fa453c42f1c0aeda9e3TREFHTMN, shape=(12776, 192, 288), dtype=float32, chunksize=(12776, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) object 1980-01-01 00:00:00 ... 2014-12-31 00:00:00

In [29]:
chunks_dict = {"time": 365}

In [28]:
12776/35

365.0285714285714

In [30]:
%%time
ds = xr.open_mfdataset(files, parallel=True, chunks= chunks_dict)
tref_min = ds['TREFHTMN']
tref_min

CPU times: user 38.1 ms, sys: 3.38 ms, total: 41.5 ms
Wall time: 91.7 ms


<xarray.DataArray 'TREFHTMN' (time: 12776, lat: 192, lon: 288)>
dask.array<open_dataset-3720f14982c801393fbaeae70835bed1TREFHTMN, shape=(12776, 192, 288), dtype=float32, chunksize=(365, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) object 1980-01-01 00:00:00 ... 2014-12-31 00:00:00

In [38]:
%%time
tnx = tref_min.groupby('time.year').max('time')
tn
tnx

CPU times: user 57.5 ms, sys: 6.52 ms, total: 64 ms
Wall time: 87.3 ms


<xarray.DataArray 'TREFHTMN' (year: 35, lat: 192, lon: 288)>
dask.array<stack, shape=(35, 192, 288), dtype=float32, chunksize=(1, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * year     (year) int64 1980 1981 1982 1983 1984 ... 2010 2011 2012 2013 2014

In [36]:
%%time
tnx = tnx.compute()
tnx

CPU times: user 548 µs, sys: 34 µs, total: 582 µs
Wall time: 588 µs


In [37]:
tnx

<xarray.DataArray 'TREFHTMN' (year: 35, lat: 192, lon: 288)>
array([[[-18.000778  , -18.000778  , -18.000778  , ..., -18.000778  ,
         -18.000778  , -18.000778  ],
        [-17.21872   , -17.280594  , -17.457687  , ..., -17.122223  ,
         -17.151505  , -17.183899  ],
        [-17.636063  , -17.74031   , -17.842285  , ..., -17.203064  ,
         -17.320831  , -17.45285   ],
        ...,
        [  0.37106323,   0.36102295,   0.34954834, ...,   0.39300537,
           0.38668823,   0.37854004],
        [  0.3157959 ,   0.30743408,   0.3034668 , ...,   0.33798218,
           0.33166504,   0.32388306],
        [  0.25976562,   0.26013184,   0.2604065 , ...,   0.25827026,
           0.25881958,   0.25930786]],

       [[-20.980545  , -20.980545  , -20.980545  , ..., -20.980545  ,
         -20.980545  , -20.980545  ],
        [-21.064117  , -21.132675  , -21.091278  , ..., -20.83754   ,
         -20.711456  , -20.841751  ],
        [-19.57109   , -19.58603   , -19.581512  , ..., -19.57396   ,
         -19.62358   , -19.504608  ],
...
        [  0.2420044 ,   0.25375366,   0.26116943, ...,   0.21240234,
           0.22183228,   0.2314148 ],
        [  0.23953247,   0.24005127,   0.2406311 , ...,   0.23562622,
           0.23849487,   0.2392273 ],
        [  0.23931885,   0.2394104 ,   0.23953247, ...,   0.2388916 ,
           0.23904419,   0.23919678]],

       [[-19.070633  , -19.070633  , -19.070633  , ..., -19.070633  ,
         -19.070633  , -19.070633  ],
        [-18.18808   , -18.15503   , -18.24884   , ..., -18.334091  ,
         -18.284302  , -18.235474  ],
        [-18.562683  , -18.41835   , -18.218323  , ..., -18.77304   ,
         -18.68164   , -18.60614   ],
        ...,
        [  0.263031  ,   0.24835205,   0.23428345, ...,   0.302063  ,
           0.28930664,   0.27694702],
        [  0.3133545 ,   0.30755615,   0.30126953, ...,   0.3267212 ,
           0.32409668,   0.31921387],
        [  0.25509644,   0.25515747,   0.255188  , ...,   0.25494385,
           0.25500488,   0.25506592]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * year     (year) int64 1980 1981 1982 1983 1984 ... 2010 2011 2012 2013 2014

In [39]:
%%time
tr = xr.where(tref_min>20,1,0).groupby('time.year').sum('time')
tr

CPU times: user 69.2 ms, sys: 3.89 ms, total: 73.1 ms
Wall time: 75.4 ms


<xarray.DataArray 'TREFHTMN' (year: 35, lat: 192, lon: 288)>
dask.array<stack, shape=(35, 192, 288), dtype=int64, chunksize=(1, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * year     (year) int64 1980 1981 1982 1983 1984 ... 2010 2011 2012 2013 2014

In [40]:
tr.compute()

<xarray.DataArray 'TREFHTMN' (year: 35, lat: 192, lon: 288)>
array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
...
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]])
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * year     (year) int64 1980 1981 1982 1983 1984 ... 2010 2011 2012 2013 2014

In [ ]:
ds[variable].groupby('time.year').max('time'), name='TNX'

In [ ]:
ds = ds.sel(time=slice('1980-01-01', '2010-12-31'))

In [10]:
for eff in range(1):
    print(eff)
    fp = files[eff]
    ds = xr.open_dataset(fp)
    ds = ds.sel(time=slice('1980-01-01', '2010-12-31'))

        ### Fixed index
    TNX[eff,:,:,:] = tempex_func.warmest_night(ds)    # Warmest night
    print('tnx')
    TNN[eff,:,:,:] = tempex_func.coolest_night(ds)    # Coolest night
    print('tnn')

        ## Fixed threshold count of days
    TR[eff,:,:,:] = tempex_func.tropical_nights(ds)   # Tropical Nights Tn>20C
    print('tr')
    FD[eff,:,:,:] = tempex_func.frost_days(ds) # Nights with TN<Oc
    print('fd')

    # Locally defined threshold
    TN90[eff,:,:,:] = tempex_func.annualnum_above_q(ds, dn90, varname='TREFHTMN')
    print('TN90')
    TN10[eff,:,:,:] = tempex_func.annualnum_below_q(ds, dn10, threshold=0.1, varname='TREFHTMN')
    print('TN10')
        

0
tnx
tnn
tr
fd
TN90
TN10


In [12]:
fileintro = 'b.e21.BSSP245smbb.f09_g17.MCB-050PCT.001-010.cam.h1.'

tnnnm = os.path.join(otnDir , (fileintro + 'TNN.' + dates+ ".nc"))
tnxnm = os.path.join(otnDir , (fileintro + 'TNX.' + dates+ ".nc"))
trnm = os.path.join(otnDir , (fileintro + 'TR.' + dates+ ".nc"))
fdnm = os.path.join(otnDir , (fileintro + 'FD.' + dates+ ".nc"))
tn90nm = os.path.join(otnDir , (fileintro + 'TN90.' + dates + ".nc"))
tn10nm = os.path.join(otnDir , (fileintro + 'TN10.' + dates+ ".nc"))

    
TNX = TNX.assign_attrs(units='Celsius',
                       longname = 'Warmest Night per year')
TNN = TNN.assign_attrs(units='Celsius',
                       longname = 'Coldest Night per year')
TR = TR.assign_attrs(units='Days per year',
                       longname = 'Tropical Nights; daily minimum above 20C')
FD = FD.assign_attrs(units='Days per year',
                       longname = 'Frost Days; daily minimum below 0C')
TN90 = TN90.assign_attrs(units='Days per year',
                       longname = 'Warm Nights above 90th Percentile')
TN10 = TN10.assign_attrs(units='Days per year',
                       longname = 'Cool Nights below 10th Percentile')


and write out to netcdf

In [13]:
TNN.to_netcdf(tnnnm)
TNX.to_netcdf(tnxnm)
TR.to_netcdf(trnm)
FD.to_netcdf(fdnm)
TN90.to_netcdf(tn90nm)
TN10.to_netcdf(tn10nm)


## Daily maximum temperature

In [ ]:
TXX = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='TXX')
TXN = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='TXN')
TX10 = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='TX10')
TX90 = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='TX90')
SU = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='SU')
ID = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='ID')

No loop for the historical data as only one model run

In [6]:
files = glob.glob(iDir + '*.TREFHTMX.*')
eff = 0
#### Open file
fp = files[eff]
ds = xr.open_dataset(fp)
ds = ds.sel(time=slice('1980-01-01', '2014-12-31'))

In [ ]:
    ### Fixed thresholds
TXX = tempex_func.warmest_day(ds)
print('txx')
TXN = tempex_func.coolest_day(ds)
print('txn')

    ## Fixed threshold count of days
SU = tempex_func.summer_days(ds) 
print('su')
ID = tempex_func.ice_days(ds) 
print('id')

In [7]:
quan10 = xr.DataArray(None, coords=dict(year=range(30), lat = lat, lon = lon), dims=dims)
quan90 = xr.DataArray(None, coords=dict(year=range(30), lat = lat, lon = lon), dims=dims)
dsclim = ds.sel(time=slice('1981-01-01','2010-12-31'))

climyears= np.arange(1981,2011,1)
# first 30 iterations drop one year at a time
for i in range(30):
    
    tdrop = dsclim.where(dsclim['time.year']!=climyears[i])
    print(climyears[i])
    quan90[i:,:,:] = tdrop.quantile(q=0.9, dim=('time'), skipna=True).squeeze().to_array()
    quan10[i:,:,:] = tdrop.quantile(q=0.1, dim=('time'), skipna=True).squeeze().to_array()

1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010


In [23]:
QX90 = quan90.mean('year')
QX10 = quan10.mean('year')

In [24]:
QX90 = QX90.to_dataset(name='QX90')
QX10 = QX10.to_dataset(name='QX10')

In [27]:
fileintro='b.e21.BWHISTcmip6.f09_g17.CMIP6-historical-WACCM.1980_2014.001.cam.h1.'

qx90nm = os.path.join(otxDir, (fileintro + 'QX90.1981-2010.nc'))
qx10nm = os.path.join(otxDir, (fileintro + 'QX10.1981-2010.nc'))

QX90 = QX90.assign_attrs(units='Degrees Celsius',
                       longname = '90th Percentile Annual Temperature - Warm Days')
QX10 = QX10.assign_attrs(units='Degrees Celsius',
                       longname = '10th Percentile Annual Temperature - Cool Days')

QX90.to_netcdf(qx90nm)
QX10.to_netcdf(qx10nm)

In [59]:
#local threshold
TX90 = tempex_func.annualnum_above_q(ds, QX90, varname='TREFHTMX')
print('TX90')
TX10 = tempex_func.annualnum_below_q(ds, QX10, threshold=0.1, varname='TREFHTMX')
print('TX10')

TX90
TX10


In [61]:
TX90 = TX90.rename({'QX90':'TX90'})
TX10 = TX10.rename({'QX10':'TX10'})

In [62]:
txxnm = os.path.join(otxDir , (fileintro + 'TXX.' + dates + '.nc'))
txnnm = os.path.join(otxDir , (fileintro + 'TXN.' + dates + '.nc'))
sunm = os.path.join(otxDir , (fileintro + 'SU.' + dates + '.nc'))
idnm = os.path.join(otxDir , (fileintro + 'ID.' + dates + '.nc'))
tx90nm = os.path.join(otxDir , (fileintro + 'TX90.' + dates + '.nc'))
tx10nm = os.path.join(otxDir , (fileintro + 'TX10.' + dates + '.nc'))

In [ ]:
TXX = TXX.assign_attrs(units='Celsius',
                       longname = 'Warmest Day per year')
TXN = TXN.assign_attrs(units='Celsius',
                       longname = 'Coldest Day per year')
SU = SU.assign_attrs(units='Days per year',
                       longname = 'Summer Days; daily maximum above 25C')
ID = ID.assign_attrs(units='Days per year',
                       longname = 'Ice Days; daily maximum below 0C')
TX90 = TX90.assign_attrs(units='Days per year',
                       longname = 'Warm Days above 90th Percentile')
TX10 = TX10.assign_attrs(units='Days per year',
                       longname = 'Cool Days below 10th Percentile')


    # and write out to netcdf
TXN.to_netcdf(txnnm)
TXX.to_netcdf(txxnm)
SU.to_netcdf(sunm)
ID.to_netcdf(idnm)
TX90.to_netcdf(tx90nm)
TX10.to_netcdf(tx10nm)